In [1]:
import PIL
from PIL import Image
import requests
from io import BytesIO
import webcolors
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import misc, ndimage
from PIL import ImageEnhance
from imageai.Detection.Custom import CustomObjectDetection

#we are using our trained model from custom image detection in order to detection the objects 
detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(
    "./final_model/models/detection_model-ex-001--loss-0031.882.h5")
detector.setJsonPath("./final_model/json/detection_config.json")
detector.loadModel()


In [45]:
'''# rename the images 
#changez ça en fonction de vos répertoires
i=1
for f in listdir("./test"):
    if isfile(join("./test", f)) : 
        os.rename(join("./test", f), join("./test",str(i)+".jpg"))
        i+=1
'''

In [2]:
#closest_color  convert the RGB values to color_name
def closest_color(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS21_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]


In [4]:
#returns the n  dominant colors
def top_colors(image, n):
    # convert the image to rgb
    image = image.convert('RGB')

    # resize the image to 300 x 300
    image = image.resize((300, 300))

    detected_colors = []
    for x in range(image.width):
        for y in range(image.height):
            detected_colors.append(closest_color(image.getpixel((x, y))))
    Series_Colors = pd.Series(detected_colors)
    output = Series_Colors.value_counts()/len(Series_Colors)
    return(output.head(n))




In [5]:
def merged_colors(color_list) : 
    for color,color_per in zip(color_list.index,color_list) : 
        if color == 'silver' : 
            try : 
                color_list['white'] += color_per
                color_list.drop(index=(color),inplace=True)
            except:
                color_list['white'] = color_per
                color_list.drop(index=(color), inplace=True)
        if color in ['olive','lime'] : 
            try : 
                color_list['green'] += color_per
                color_list.drop(index=(color), inplace=True)
            except : 
                color_list['green'] = color_per
                color_list.drop(index=(color), inplace=True)
        if color in ['aqua','teal','navy'] :
            try : 
                color_list['blue'] += color_per
                color_list.drop(index=(color), inplace=True)
            except :
                color_list['blue'] = color_per
                color_list.drop(index=(color), inplace=True)
        if color in ['maroon']:
            try : 
                color_list['red'] += color_per
                color_list.drop(index=(color), inplace=True)
            except:
                color_list['red'] = color_per
                color_list.drop(index=(color), inplace=True)
    return color_list

In [6]:
#returns the pourcentage of drawn area and its position
def exp_area(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(
        blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    x, y, w, h = cv2.boundingRect(thresh)
    return round((w*h)/(image.shape[0]*image.shape[1])*100, 2), round((x)/(image.shape[0])*100, 2), round((y)/(image.shape[1])*100, 2)
    


In [7]:
#returns the features of a drawing
def features(image) :
    print("traitement sur : "+image)
    img = Image.open(image)
    colors = merged_colors(top_colors(img, 6))
    print('Colors : ' ,colors)
    img = cv2.imread(image)
    area, x, y = exp_area(img)
    print('Exploited area : ' , area)
    position = [x, y]
    print('Position : ' , position)
    objects = detection(image)
    print('Objects : ', objects)
    return colors,position, area,objects


In [8]:
# returns the detected objects
def detection(image):
    output_image = '../'+image[0:-4]+'_rest.jpg'
    detections = detector.detectObjectsFromImage(
        input_image=image, output_image_path=output_image, minimum_percentage_probability=80)
    return list(map(lambda n: n['name'], detections))


In [9]:
#classify a drawing depending on its extracted features
def classifier(colors,position,exp_area,objects) : 
    if (exp_area < 30 and ((position[0] < 25 or position[0] > 75) and (position[1] < 25 or position[1] > 75))):
        print('Introversion and timidity')
        return 'Introversion and timidity'
    if 'purple' in colors.index and colors['purple'] > 0.25:
        print('Tension and emotional conflicts')
        return 'Tension and emotional conflicts'
    indice_p = 0
    for color in  colors.index : 
        if color in ['blue', 'green', 'yellow', 'Fuchsia'] and colors[color] > 0.01:
            
            indice_p += 1
    for object_ in objects : 
        if object_ in ['sun', 'rainbow']:
            indice_p += 1
    indice_n = 0 
    for color in colors.index:
        if color in ['red', 'black'] and colors[color] > 0.02:
            indice_n += 1
    for object_ in objects:
        if object_ in ['knife', 'rocket', 'hammer']:
            indice_n += 2
    if indice_n  > indice_p :
        print('Aggression and hostility')
        return ('Aggression and hostility')
    elif indice_n <= indice_p:
        print('Calm, peace and balance')
        return ('Calm, peace and balance')
    return print('Model failed to detect any emotional state')


In [83]:
#initialisation 
list_outputs = []
#list_white_parts = []
list_contours = []
list_exp_areas=[]
list_positions=[]
list_objects =[]
list_class_ = []
#récupère la listes de noms de fichier
list_images = [f for f in listdir("./test") if isfile(join("./test", f))]


In [84]:
#feature extraction for a list of drawings
for image in list_images:
    print("traitement sur : "+image)
    img = Image.open(join('./test/'+image))
    color = merged_colors(top_colors(img, 5))
    list_outputs.append(color)
    img = cv2.imread(join('./test/'+image))
    area, x, y = exp_area(img)
    position=[x,y]
    list_exp_areas.append(area)
    list_positions.append([x,y])
    objects = detection(join('./test/'+image))
    list_objects.append(objects)
    list_class_.append(classifier(color,position,area,objects))


traitement sur : 1.jpg
[620, 61, 784, 234]
[10, 354, 127, 547]
Calm, peace and balance
traitement sur : 10.jpg
[21, 159, 88, 322]
[23, 170, 86, 325]
[313, 201, 382, 342]
[165, 232, 246, 338]
[158, 248, 267, 316]
[313, 223, 375, 333]
Calm, peace and balance
traitement sur : 11.jpg
[122, 348, 279, 635]
Calm, peace and balance
traitement sur : 12.jpg
[219, 0, 551, 127]
[0, 9, 272, 240]
[337, 153, 493, 449]
[474, 194, 666, 405]
[80, 329, 204, 555]
[80, 329, 204, 555]
Calm, peace and balance
traitement sur : 13.jpg
[695, 0, 1021, 520]
[130, 181, 385, 714]
[863, 482, 1005, 836]
[576, 675, 611, 725]
[426, 757, 459, 845]
Aggression and hostility
traitement sur : 14.jpg
[662, 177, 762, 883]
[662, 177, 762, 883]
[462, 312, 592, 827]
[293, 464, 473, 865]
[810, 372, 937, 953]
Calm, peace and balance
traitement sur : 15.jpg
[201, 0, 337, 332]
[13, 18, 155, 275]
Calm, peace and balance
traitement sur : 16.jpg
[521, 0, 833, 293]
[419, 12, 939, 273]
[45, 66, 510, 776]
[359, 333, 922, 719]
Aggression a

In [93]:
#putting the previous results on a dataframe
dict_images = {}
'''dict_images.update({"Image": list_images, "color": list_outputs,
                   "white_part": list_white_parts, "contour": list_contours,"exp_area":list_exp_areas , 'position' : list_positions})
'''
dict_images.update({"image": list_images, "color": list_outputs, 'position': list_positions,
                   "exp_area": list_exp_areas , 'objects' : list_objects,'class' : list_class_})


In [94]:
df = pd.DataFrame(dict_images)
df

,image,color,position,exp_area,objects,class
0,1.jpg,white 0.842422 yellow 0.082278 orange ...,"[3.0, 7.0]",84.89,"[person, person]","Calm, peace and balance"
1,10.jpg,white 0.865233 gray 0.055433 yellow ...,"[0.76, 4.2]",91.96,"[person, tree, person, person, tree, tree]","Calm, peace and balance"
2,11.jpg,white 0.994867 gray 0.001422 green 0...,"[22.6, 31.21]",2.56,[person],"Calm, peace and balance"
3,12.jpg,white 0.706800 gray 0.068756 yellow ...,"[0.0, 0.0]",100.00,"[sun, person, person, person, person, tree]","Calm, peace and balance"
4,13.jpg,white 0.695678 gray 0.158511 orange ...,"[0.0, 0.0]",98.73,"[person, rocket, person, flower, flower]",Aggression and hostility
5,14.jpg,white 0.932867 yellow 0.023789 gray ...,"[0.0, 0.77]",97.76,"[person, tree, person, person, person]","Calm, peace and balance"
6,15.jpg,gray 0.131911 white 0.816144 black 0...,"[0.0, 0.98]",96.14,"[tree, tree]","Calm, peace and balance"
7,16.jpg,white 0.557367 gray 0.224611 black 0...,"[0.0, 0.0]",97.57,"[person, sun, rocket, house]",Aggression and hostility
8,17.jpg,white 0.961011 orange 0.018811 gray ...,"[1.85, 19.25]",56.90,"[person, person, tree, tree]","Calm, peace and balance"
9,18.jpg,white 0.898689 gray 0.078489 black 0...,"[3.45, 1.86]",90.53,"[tree, rocket, person]",Aggression and hostility


In [64]:
#reading the csv file of the classified drawings (by the psychologist)
pd.read_csv('./images.csv')


,image,class
0,1.jpg,"Calm, peace and balance"
1,10.jpg,"Calm, peace and balance"
2,11.jpg,"Calm, peace and balance"
3,12.jpg,"Calm, peace and balance"
4,13.jpg,Aggression and hostility
5,14.jpg,"Calm, peace and balance"
6,15.jpg,"Calm, peace and balance"
7,16.jpg,"Calm, peace and balance"
8,17.jpg,"Calm, peace and balance"
9,18.jpg,Aggression and hostility


In [98]:
compt=0
for (class1,class2) in zip(df_['class'],df['class']) : 
    if class1!= class2 :
        compt +=1
acc= (60-compt)/60

In [99]:
#print the accuracy 
acc

0.9666666666666667